In [1]:
import collections
import math
import os
import random
import zipfile
import numpy as np
import tensorflow as tf

In [2]:
def read_data(filename):
    with zipfile.ZipFile(filename) as f:
        data = tf.compat.as_str(f.read(f.namelist()[0])).split()
    return data

vocabulary = read_data("text8.zip")
print('Data size', len(vocabulary))

Data size 17005207


In [3]:
vocabulary_size = 50000

def build_dataset(words, n_words):
    """Process raw inputs into a dataset."""
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(n_words - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary
data, count, dictionary, reverse_dictionary = build_dataset(vocabulary,
                                                            vocabulary_size)

In [4]:
data_index = 0
def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1  # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span)
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    for i in range(batch_size // num_skips):
        target = skip_window  # target label at the center of the buffer
        targets_to_avoid = [skip_window]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[target]
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    # Backtrack a little bit to avoid skipping words in the end of a batch
    data_index = (data_index + len(data) - span) % len(data)
    return batch, labels

batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=1)

In [5]:
# Step 4: Build and train a skip-gram model.

batch_size = 128
embedding_size = 128  # Dimension of the embedding vector.
skip_window = 1       # How many words to consider left and right.
num_skips = 2         # How many times to reuse an input to generate a label.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64    # Number of negative examples to sample.

graph = tf.Graph()

with graph.as_default():

  # Input data.
  train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
  train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
  valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

  # Ops and variables pinned to the CPU because of missing GPU implementation
  with tf.device('/cpu:0'):
    # Look up embeddings for inputs.
    embeddings = tf.Variable(
        tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    embed = tf.nn.embedding_lookup(embeddings, train_inputs)

    # Construct the variables for the NCE loss
    nce_weights = tf.Variable(
        tf.truncated_normal([vocabulary_size, embedding_size],
                            stddev=1.0 / math.sqrt(embedding_size)))
    nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

  # Compute the average NCE loss for the batch.
  # tf.nce_loss automatically draws a new sample of the negative labels each
  # time we evaluate the loss.
  loss = tf.reduce_mean(
      tf.nn.nce_loss(weights=nce_weights,
                     biases=nce_biases,
                     labels=train_labels,
                     inputs=embed,
                     num_sampled=num_sampled,
                     num_classes=vocabulary_size))

  # Construct the SGD optimizer using a learning rate of 1.0.
  optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

  # Compute the cosine similarity between minibatch examples and all embeddings.
  norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
  normalized_embeddings = embeddings / norm
  valid_embeddings = tf.nn.embedding_lookup(
      normalized_embeddings, valid_dataset)
  similarity = tf.matmul(
      valid_embeddings, normalized_embeddings, transpose_b=True)

  # Add variable initializer.
  init = tf.global_variables_initializer()





In [6]:
# Step 5: Begin training.
num_steps = 2000001

with tf.Session(graph=graph) as session:
  # We must initialize all variables before we use them.
  init.run()
  print('Initialized')

  average_loss = 0
  for step in range(num_steps):
    batch_inputs, batch_labels = generate_batch(
        batch_size, num_skips, skip_window)
    feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

    # We perform one update step by evaluating the optimizer op (including it
    # in the list of returned values for session.run()
    _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
    average_loss += loss_val

    if step % 20000 == 0:
      if step > 0:
        average_loss /= 20000
      # The average loss is an estimate of the loss over the last 2000 batches.
      print('Average loss at step ', step, ': ', average_loss)
      average_loss = 0

    # Note that this is expensive (~20% slowdown if computed every 500 steps)
    if step % 200000 == 0:
      sim = similarity.eval()
      for i in range(valid_size):
        valid_word = reverse_dictionary[valid_examples[i]]
        top_k = 8  # number of nearest neighbors
        nearest = (-sim[i, :]).argsort()[1:top_k + 1]
        log_str = 'Nearest to %s:' % valid_word
        for k in range(top_k):
          close_word = reverse_dictionary[nearest[k]]
          log_str = '%s %s,' % (log_str, close_word)
        print(log_str)
  final_embeddings = normalized_embeddings.eval()


Initialized
Average loss at step  0 :  285.415283203
Nearest to from: leh, gemological, lights, jujutsu, cages, bride, dives, confidence,
Nearest to into: musically, occlusion, pathologist, grothendieck, by, mcneil, lud, fredric,
Nearest to been: illustrators, judson, overshadowed, doubt, musab, hellenism, roaring, sato,
Nearest to and: maududi, zoot, poker, posed, benz, halfwidth, solvent, broken,
Nearest to which: devoted, mackenzie, fullwidth, originality, separable, plaintexts, churches, icsu,
Nearest to while: woolwich, disguise, dayton, metrical, pond, neutrino, especially, pottery,
Nearest to between: whistleblower, bushman, thriller, distinguishing, genotype, uruguayan, trombone, emulation,
Nearest to he: seriousness, grantley, apsu, raiding, hani, troughs, unforgettable, indoctrination,
Nearest to if: bryson, awe, dominion, heathland, demographics, negating, adhesion, cannabis,
Nearest to to: encompassed, axel, jacopo, mr, tain, poetic, compute, necromancers,
Nearest to were: 

Average loss at step  820000 :  4.13990458595
Average loss at step  840000 :  4.14843492368
Average loss at step  860000 :  4.16720801257
Average loss at step  880000 :  4.11716017977
Average loss at step  900000 :  4.16926217705
Average loss at step  920000 :  4.14734866341
Average loss at step  940000 :  4.12868653712
Average loss at step  960000 :  4.14620680249
Average loss at step  980000 :  4.13878755189
Average loss at step  1000000 :  4.04580348073
Nearest to from: through, via, rajons, during, across, besides, into, enhances,
Nearest to into: through, within, across, onto, tempest, from, davidic, affectionate,
Nearest to been: be, become, come, already, were, remained, was, recently,
Nearest to and: but, rajonas, or, agave, thibetanus, dasyprocta, hyi, landesverband,
Nearest to which: that, this, what, typically, actually, who, safl, but,
Nearest to while: although, when, though, however, whereas, kinderhook, unlike, whilst,
Nearest to between: lup, among, within, with, across

Average loss at step  1820000 :  4.0366222593
Average loss at step  1840000 :  4.01050049885
Average loss at step  1860000 :  4.04104860071
Average loss at step  1880000 :  4.02129215995
Average loss at step  1900000 :  4.03252021322
Average loss at step  1920000 :  4.07884682775
Average loss at step  1940000 :  4.00356332458
Average loss at step  1960000 :  4.07981533021
Average loss at step  1980000 :  4.03165701723
Average loss at step  2000000 :  4.02683587987
Nearest to from: via, besides, through, rajons, in, safl, during, handicap,
Nearest to into: onto, through, within, across, davidic, tempest, from, beyond,
Nearest to been: be, become, already, grown, fallen, come, historically, undergone,
Nearest to and: but, nor, or, rajonas, however, dasyprocta, UNK, hyi,
Nearest to which: that, this, typically, what, actually, but, safl, islamists,
Nearest to while: although, whilst, whereas, when, though, however, despite, including,
Nearest to between: lup, among, with, within, across, 

In [7]:
# Step 6: Visualize the embeddings.


def plot_with_labels(low_dim_embs, labels, filename='tsne.png'):
  assert low_dim_embs.shape[0] >= len(labels), 'More labels than embeddings'
  plt.figure(figsize=(18, 18))  # in inches
  for i, label in enumerate(labels):
    x, y = low_dim_embs[i, :]
    plt.scatter(x, y)
    plt.annotate(label,
                 xy=(x, y),
                 xytext=(5, 2),
                 textcoords='offset points',
                 ha='right',
                 va='bottom')

  plt.savefig(filename)

try:
  # pylint: disable=g-import-not-at-top
  from sklearn.manifold import TSNE
  import matplotlib.pyplot as plt

  tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)
  plot_only = 500
  low_dim_embs = tsne.fit_transform(final_embeddings[:plot_only, :])
  labels = [reverse_dictionary[i] for i in range(plot_only)]
  plot_with_labels(low_dim_embs, labels)

except ImportError:
  print('Please install sklearn, matplotlib, and scipy to show embeddings.')